In [1]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# load text
raw_text = load_doc('data/shakespeare.txt')
# clean
raw_text = ''.join([i for i in raw_text if not i.isdigit()])
tokens = raw_text.split()
raw_text = ' '.join(tokens)

Using TensorFlow backend.


In [2]:
#organize into sequences of characters
length = 40
n = 1
sequences = list()
for i in range(length, len(raw_text), n):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
    #print('Total Sequences: %d' % len(sequences))

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()    
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

In [3]:
#load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

In [4]:
sequences = list()
for line in lines:
    #integer encode line
    encoded_seq = [mapping[char] for char in line]
    #store
    sequences.append(encoded_seq)
#vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 61


In [5]:
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)
X.shape

(93633, 40, 61)

In [7]:
# define the model
def define_model(X):
    model = Sequential()
    model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(vocab_size, activation='softmax'))
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    model.summary()
   # plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [8]:
# define model
model = define_model(X)
filepath="shakes-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
callbacks = [EarlyStopping(monitor='loss', min_delta=0.005, patience=2),
             ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, mode='min')]


#model.fit(X, y, epochs=500, verbose=2, callbacks=callbacks)
model.fit(X, y, epochs=100, verbose=2, callbacks=callbacks)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 150)               127200    
_________________________________________________________________
dense_1 (Dense)              (None, 61)                9211      
Total params: 136,411
Trainable params: 136,411
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
 - 279s - loss: 2.3745 - accuracy: 0.3337

Epoch 00001: saving model to shakes-weights-improvement-01-2.3745.hdf5
Epoch 2/100
 - 260s - loss: 2.0093 - accuracy: 0.4160

Epoch 00002: saving model to shakes-weights-improvement-02-2.0093.hdf5
Epoch 3/100
 - 245s - loss: 1.8798 - accuracy: 0.4455

Epoch 00003: saving model to shakes-weights-improvement-03-1.8798.hdf5
Epoch 4/100
 - 259s - loss: 1.7875 - accuracy: 0.4673

Epoch 00004: saving model to shakes-weights-improvement-04-1.7875.hdf5
Epoch 

In [9]:
# save the model to file
model.save('model_upper_56.h5')
# save the mapping
dump(mapping, open('mapping_upper.pkl', 'wb'))